In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings("ignore")

In [2]:
%matplotlib inline

In [2]:
spotify_df = pd.read_csv('data/spotify_data.csv')

In [ ]:
spotify_df.head()

In [5]:
spotify_df.dtypes

Unnamed: 0            int64
artist_name          object
track_name           object
track_id             object
popularity            int64
year                  int64
genre                object
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
duration_ms           int64
time_signature        int64
dtype: object

In [5]:
# Check if this is a list or a string that looks like a list
spotify_df['genre'].values

array(['acoustic', 'acoustic', 'acoustic', ..., 'trip-hop', 'trip-hop',
       'trip-hop'], dtype=object)

Feature Engineering

In [6]:
spotify_df.tail()

,Unnamed: 0,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
1159759,1473391,Nicola Conte,Black Spirits,0m27F0IGHLGAWhqd6ccYst,4,2011,trip-hop,0.373,0.742,10,-6.453,0,0.0736,0.325,0.000141,0.1590,0.5220,107.951,344013,3
1159760,1473392,Nicola Conte,Quiet Dawn,6er9p611eHEcUCU50j7D57,3,2011,trip-hop,0.516,0.675,7,-7.588,0,0.0326,0.788,0.000129,0.1300,0.2640,119.897,285067,4
1159761,1473393,Amon Tobin,Morning Ms Candis,7jsMMqxy1tt0rH5FzYcZTQ,2,2011,trip-hop,0.491,0.440,5,-8.512,1,0.0274,0.477,0.003130,0.0936,0.0351,100.076,214253,4
1159762,1473394,Peace Orchestra,Happy Christmas (War Is Over),77lA1InUaXztuRk2vOzD1S,0,2011,trip-hop,0.480,0.405,0,-13.343,1,0.0276,0.431,0.000063,0.1250,0.2020,133.885,239133,3
1159763,1473395,Mo' Horizons,Hit the Road Jack (Pé Na Éstrada),4oMiOwhDZEdBuzAfhzRHbi,3,2011,trip-hop,0.782,0.861,1,-7.292,0,0.1250,0.220,0.000008,0.0581,0.8570,89.987,212227,4


In [3]:
float_cols = spotify_df.dtypes[spotify_df.dtypes == 'float64'].index
print(float_cols)

Index(['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo'],
      dtype='object')


In [4]:
ohe_cols = 'popularity'

In [11]:
# Change display options to avoid scientific notation
pd.options.display.float_format = '{:.2f}'.format

# describe(): This method is used to generate descriptive statistics that summarize the central tendency, dispersion, 
# and shape of a dataset’s distribution, excluding NaN values. 
spotify_df['popularity'].describe()

count   1159764.00
mean         18.38
std          15.89
min           0.00
25%           5.00
50%          15.00
75%          29.00
max         100.00
Name: popularity, dtype: float64

In [5]:
# creates a new column popularity_transform by scaling down the values of the popularity column and converting them to integers. 
# Each original popularity score is divided by 5, and the result is truncated to an integer. 
spotify_df['popularity_transform'] = spotify_df['popularity'].apply(lambda x: int(x / 5))

# Get the position of the 'popularity' column
popularity_index = spotify_df.columns.get_loc('popularity')

# Insert the 'popularity_transform' column next to the 'popularity' column
spotify_df.insert(popularity_index + 1, 'popularity_transform', spotify_df.pop('popularity_transform'))

In [ ]:
spotify_df.head()

In [13]:
# Since tfidf cannot handle nulls, check if there are any null values in the entire DataFrame
any_null = spotify_df.isnull().values.any()
print(f"Any null values in DataFrame: {any_null}")

# Count of null values in each column
null_counts = spotify_df.isnull().sum()
print("\nCount of null values in each column:")
print(null_counts)

Any null values in DataFrame: True

Count of null values in each column:
Unnamed: 0               0
artist_name             15
track_name               1
track_id                 0
popularity               0
popularity_transform     0
year                     0
genre                    0
danceability             0
energy                   0
key                      0
loudness                 0
mode                     0
speechiness              0
acousticness             0
instrumentalness         0
liveness                 0
valence                  0
tempo                    0
duration_ms              0
time_signature           0
dtype: int64


In [6]:
spotify_df['artist_name'] = spotify_df['artist_name'].apply(lambda d: d if isinstance(d, str) else '')
spotify_df['track_name'] = spotify_df['track_name'].apply(lambda d: d if isinstance(d, str) else '')


In [15]:
# Check if we eliminated the null values
any_null = spotify_df.isnull().values.any()
print(f"Any null values in DataFrame: {any_null}")

# Count of null values in each column
null_counts = spotify_df.isnull().sum()
print("\nCount of null values in each column:")
print(null_counts)

Any null values in DataFrame: False

Count of null values in each column:
Unnamed: 0              0
artist_name             0
track_name              0
track_id                0
popularity              0
popularity_transform    0
year                    0
genre                   0
danceability            0
energy                  0
key                     0
loudness                0
mode                    0
speechiness             0
acousticness            0
instrumentalness        0
liveness                0
valence                 0
tempo                   0
duration_ms             0
time_signature          0
dtype: int64


In [7]:
def create_one_hot_encoded_features(df, column, new_name): 
    """ 
    Create One Hot Encoded features for a specific column in the DataFrame.

    This function takes a specified column from the DataFrame, applies one-hot encoding to convert categorical values into a binary matrix, 
    and renames the resulting columns with a new prefix. The new columns will be prefixed with the provided new_name, followed by the original column value.

    Parameters: 
        df (pandas.DataFrame): The input DataFrame containing the data.
        column (str): The name of the column to be one-hot encoded.
        new_name (str): The prefix to be used for the new one-hot encoded columns.
        
    Returns: 
        pandas.DataFrame: A DataFrame containing the one-hot encoded features with the new column names.
    """

    # Apply one-hot encoding to the specified column
    transformed_df = pd.get_dummies(df[column])

    # Get the names of the newly created one-hot encoded columns
    feature_names = transformed_df.columns

    # Rename the columns with the new prefix
    transformed_df.columns = [new_name + "|" + str(i) for i in feature_names]

    # Reset the index of the DataFrame
    transformed_df.reset_index(drop = True, inplace = True)   
     
    return transformed_df

TF-IDF Section

Term Frequency Inverse Document Frequency. This is very common algorithm to transform text into a meaningful representation of numbers which is used to fit machine algorithm for prediction. It is a statistical formula to convert text documents into vectors based on the relevancy of the word. It is based on the bag of the words model to create a matrix containing the information about less relevant and most relevant words in the document.

In [22]:
# This function used tf-idf in genre. This might be suited for "all liked songs" but not for specific playlists since the playlist is already grouped by genre.
def create_feature_set(df, float_cols):
    """ 
    Process Spotify DataFrame to create a final set of features for generating recommendations.

    This function performs several steps to preprocess the given DataFrame and generate a set of features that can be used for building a recommendation system. The steps include:
    1. Formatting the 'genre' column.
    2. Applying TF-IDF vectorization to the 'genre' column to convert text data into numerical features.
    3. One-hot encoding the 'year' and 'popularity_transform' columns.
    4. Scaling specified float columns.
    5. Concatenating all processed features into a single DataFrame.
    6. Adding the 'track_id' column to maintain track identifiers.

    Parameters: 
        df (pandas.DataFrame): The Spotify DataFrame containing track data.
        float_cols (list of str): List of columns with float values that need to be scaled.

    Returns: 
        pandas.DataFrame: A DataFrame containing the final set of features for each track.
    """

    # Ensure 'genre' column is properly formatted as a string
    df['genre'] = df['genre'].apply(lambda x: str(x) if isinstance(x, str) else '')
    
    # Apply TF-IDF Vectorizer to the 'genre' column
    tfidf = TfidfVectorizer()
    processed_genres = df['genre']

    # Check for empty genre documents after preprocessing
    if processed_genres.str.strip().eq('').any():
        raise ValueError("Some documents are empty after preprocessing. Check the 'genre' column.")

    # Transform genres into TF-IDF features
    tfidf_matrix = tfidf.fit_transform(processed_genres)
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]
    genre_df.reset_index(drop=True, inplace=True)

    # One-hot encode the 'year' and 'popularity_transform' columns, scaling their impact
    year_ohe = create_one_hot_encoded_features(df, 'year','year') * 0.5
    popularity_ohe = create_one_hot_encoded_features(df, 'popularity_transform','pop') * 0.15

    # Scale specified float columns
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    # Concatenate all features into a single DataFrame
    final_df = pd.concat([genre_df, floats_scaled, popularity_ohe, year_ohe], axis = 1)
     
    # Add the 'track_id' to the final DataFrame
    final_df['track_id'] = df['track_id'].values
    
    return final_df

In [23]:
complete_feature_set = create_feature_set(spotify_df, float_cols = float_cols)

In [67]:
def test_feature_set_nogenre_year_popularity(df, float_cols):
    """ 
    Process Spotify DataFrame to create a final set of features for generating recommendations.

    This function performs several steps to preprocess the given DataFrame and generate a set of features that can be used for building a recommendation system. The steps include:
    1. Formatting the 'genre' column.
    2. Applying TF-IDF vectorization to the 'genre' column to convert text data into numerical features.
    3. One-hot encoding the 'year' and 'popularity_transform' columns.
    4. Scaling specified float columns.
    5. Concatenating all processed features into a single DataFrame.
    6. Adding the 'track_id' column to maintain track identifiers.

    Parameters: 
        df (pandas.DataFrame): The Spotify DataFrame containing track data.
        float_cols (list of str): List of columns with float values that need to be scaled.

    Returns: 
        pandas.DataFrame: A DataFrame containing the final set of features for each track.
    """

    # Ensure 'genre' column is properly formatted as a string
    # df['genre'] = df['genre'].apply(lambda x: str(x) if isinstance(x, str) else '')
    
    # # Apply TF-IDF Vectorizer to the 'genre' column
    # tfidf = TfidfVectorizer()
    # processed_genres = df['genre']

    # # Check for empty genre documents after preprocessing
    # if processed_genres.str.strip().eq('').any():
    #     raise ValueError("Some documents are empty after preprocessing. Check the 'genre' column.")

    # # Transform genres into TF-IDF features
    # tfidf_matrix = tfidf.fit_transform(processed_genres)
    # genre_df = pd.DataFrame(tfidf_matrix.toarray())
    # genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]
    # genre_df.reset_index(drop=True, inplace=True)

    # # One-hot encode the 'year' and 'popularity_transform' columns, scaling their impact
    # year_ohe = create_one_hot_encoded_features(df, 'year','year') * 0.5
    # popularity_ohe = create_one_hot_encoded_features(df, 'popularity_transform','pop') * 0.15

    # Scale specified float columns
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    # Concatenate all features into a single DataFrame
    # final_df = pd.concat([genre_df, floats_scaled, popularity_ohe, year_ohe], axis = 1)
    final_df = pd.concat([floats_scaled], axis = 1)
     
    # Add the 'track_id' to the final DataFrame
    final_df['track_id'] = df['track_id'].values
    
    return final_df

In [73]:
test_feature_set = test_feature_set_nogenre_year_popularity(spotify_df, float_cols=float_cols)

In [74]:
from IPython.display import display, HTML
display(HTML(test_feature_set.tail().to_html()))

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_id
1159759,0.075126,0.1484,0.160714,0.015160,0.065261,0.000028,0.03180,0.10440,0.086363,0m27F0IGHLGAWhqd6ccYst
1159760,0.103927,0.1350,0.157182,0.006715,0.158233,0.000026,0.02600,0.05280,0.095920,6er9p611eHEcUCU50j7D57
1159761,0.098892,0.0880,0.154307,0.005644,0.095783,0.000626,0.01872,0.00702,0.080063,7jsMMqxy1tt0rH5FzYcZTQ
1159762,0.096677,0.0810,0.139274,0.005685,0.086546,0.000013,0.02500,0.04040,0.107111,77lA1InUaXztuRk2vOzD1S
1159763,0.157503,0.1722,0.158103,0.025747,0.044177,0.000002,0.01162,0.17140,0.071992,4oMiOwhDZEdBuzAfhzRHbi


Spotify API

In [26]:
import os
import sys
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyPKCE

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()  

spotify_client_id = os.getenv('CLIENT_ID')
spotify_client_secret = os.getenv('CLIENT_SECRET')

In [27]:
# The set of scopes you set during the authorization, determines the access permissions that the user is asked to grant.
# scope = "playlist-read-private playlist-modify-private playlist-modify-public user-read-playback-position user-top-read user-read-recently-played user-library-read"
scope = ["user-library-read", "playlist-read-private", "playlist-modify-private"]

In [19]:
if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

In [ ]:

# Authorization Code with PKCE since I will not be using a server. I will have to implement Authorization Code with PKCE if I want
# to expose this in a web app.
 
# PKCE
# __init__(client_id=None, redirect_uri=None, state=None, scope=None, cache_path=None, username=None, proxies=None, 
#          requests_timeout=None, requests_session=True, open_browser=True, cache_handler=None)
# auth_manager = SpotifyPKCE(spotify_client_id, 'https://localhost:8888/callback', scope=scope)

auth_manager = SpotifyOAuth(spotify_client_id, spotify_client_secret, 'https://localhost:8888/callback', scope=scope, show_dialog=True)
# Generate the authorization URL
auth_url = auth_manager.get_authorize_url()

# Open the authorization URL in the default browser
print(f'Please navigate to the following URL to authorize the application: {auth_url}')


In [85]:
# Manually enter the full redirected URL after authorization
redirected_url = input("")

# Extract the authorization code from the URL
code = auth_manager.parse_response_code(redirected_url)

# Obtain the access token using the authorization code
token_info = auth_manager.get_access_token(code)

# Initialize Spotify client with the access token
sp = spotipy.Spotify(auth=token_info['access_token'])

# token = util.prompt_for_user_token(scope, client_id=spotify_client_id, client_secret=spotify_client_secret, redirect_uri='https://localhost:8888/callback')

In [25]:
# The authorization code flow is suitable for long-running applications (e.g. web and mobile apps) where the user grants permission only once.
# If you’re using the authorization code flow in a mobile app, or any other type of application where the client secret can't be safely stored, then you should use the PKCE extension.
# This flow is required for accessing user-specific data, such as the user's library, playlists, and other private data that the user has given permission to access.

# sp = spotipy.Spotify(auth=token)

# Note: Both types of authentication are not needed on an actual app.

In [ ]:
# Gather playlists names and images

id_name = {}
list_photo = {}

for i in sp.current_user_playlists(50)['items']:
    # print(i)
    id_name[i['name']] = i['uri'].split(':')[2]
    # list_photo[i['uri'].split(':')[2]] = i['images'][0]['url'] 

    # Check if 'images' exists and is not None
    if i['images']:
        list_photo[i['uri'].split(':')[2]] = i['images'][0]['url']
    else:
        list_photo[i['uri'].split(':')[2]] = None 

id_name


In [31]:
def create_playlist_dataframe(playlist_name, id_dic, df, limit=100):
    """
    This function retrieves songs from a specified playlist using the Spotify API, filters the songs to include only those available in 
    the provided Kaggle dataset, and returns the filtered playlist. The resulting playlist DataFrame includes artist names, track names, track IDs, 
    album image URLs, and the dates when the tracks were added to the playlist.

    Parameters: 
        playlist_name (str): The name of the playlist to pull from the Spotify API.
        id_dic (dict): A dictionary mapping playlist names to their corresponding playlist IDs.
        df (pandas.DataFrame): The DataFrame containing the Kaggle dataset with track information.
        
    Returns: 
        pandas.DataFrame: A DataFrame containing all songs in the specified playlist that are available in the Kaggle dataset.
    """
    
    # Initialize an empty DataFrame for the playlist
    playlist = pd.DataFrame()
    # playlist_items(playlist_id, fields=None, limit=100, offset=0, market=None, additional_types=('track', 'episode'))
    offset = 0
    tracks = []

    while True:
        response = sp.playlist_items(id_dic[playlist_name], limit=limit, offset=offset)
        items = response['items']

        if not items:
            break

        tracks.extend(items)
        offset += len(items)

    # Retrieve tracks from the specified playlist using the Spotify API
    # for index, i in enumerate(sp.playlist(id_dic[playlist_name])['tracks']['items']):
    
    for index, i in enumerate(tracks):
        # Extract and assign track details to the DataFrame
        playlist.loc[index, 'artist'] = i['track']['artists'][0]['name']
        playlist.loc[index, 'name'] = i['track']['name']
        playlist.loc[index, 'id'] = i['track']['id']
        playlist.loc[index, 'url'] = i['track']['album']['images'][1]['url']
        playlist.loc[index, 'date_added'] = i['added_at']

    # Convert the 'date_added' column to datetime format
    playlist['date_added'] = pd.to_datetime(playlist['date_added'])

    # Rename the 'id' column to 'track_id'
    playlist.rename(columns={'id': 'track_id'}, inplace=True)

    # Filter the playlist to include only songs that are available in the Kaggle dataset
    playlist = playlist[playlist['track_id'].isin(df['track_id'].values)].sort_values('date_added', ascending=False)
    
    return playlist

In [32]:
def create_liked_songs_dataframe(spotify_data, limit=50):
    """
    This function retrieves the user's liked songs using the Spotify API. The function filters the songs to include only those available in 
    the provided Kaggle dataset, and returns the filtered playlist. The resulting playlist DataFrame includes artist names, track names, track IDs, 
    album image URLs, and the dates when the tracks were added to the playlist.

    Parameters: 
        spotify_data (pandas.DataFrame): The DataFrame containing the Kaggle dataset with track information.
        limit (int): The maximum number of songs to retrieve from the user's liked songs playlist. The maximum limit is 50 per call.
        
    Returns: 
        pandas.DataFrame: A DataFrame containing all songs in the specified playlist that are available in the Kaggle dataset.
    """
    # Initialize an empty DataFrame for the most_liked_playlist
    most_liked_playlist = pd.DataFrame()
    offset = 0
    tracks = []

    while True:
        response = sp.current_user_saved_tracks(limit=limit, offset=offset)
        items = response['items']

        if not items:
            break

        tracks.extend(items)
        offset += len(items)
    
    # Retrieve "liked" tracks from the specified most_liked_playlist using the Spotify API
    # The maximum songs it can retrieve is 50
    
    for index, i in enumerate(tracks):
        # Extract and assign track details to the DataFrame
        most_liked_playlist.loc[index, 'artist'] = i['track']['artists'][0]['name']
        most_liked_playlist.loc[index, 'name'] = i['track']['name']
        most_liked_playlist.loc[index, 'id'] = i['track']['id']
        most_liked_playlist.loc[index, 'url'] = i['track']['album']['images'][1]['url']
        most_liked_playlist.loc[index, 'date_added'] = i['added_at']

    # Convert the 'date_added' column to datetime format
    most_liked_playlist['date_added'] = pd.to_datetime(most_liked_playlist['date_added'])

    # Rename the 'id' column to 'track_id'
    most_liked_playlist.rename(columns={'id': 'track_id'}, inplace=True)

    # Filter the most_liked_playlist to include only songs that are available in the Kaggle dataset
    most_liked_playlist = most_liked_playlist[most_liked_playlist['track_id'].isin(spotify_data['track_id'].values)].sort_values('date_added', ascending=False)
    
    return most_liked_playlist

In [27]:
liked_songs = create_liked_songs_dataframe(spotify_df)

In [28]:
liked_songs.shape

(112, 5)

In [86]:
chosen_user_playlist = create_playlist_dataframe('Classical', id_name, spotify_df)

In [87]:
chosen_user_playlist.head()

,artist,name,track_id,url,date_added
31,Johann Sebastian Bach,"Sonata in E-Flat Major, BWV 1031: II. Siciliano",0Q1zWiGYIPyN3TVRkUnukU,https://i.scdn.co/image/ab67616d00001e028a5465...,2024-06-10 13:46:51+00:00
30,Johann Sebastian Bach,"Sonata in G Minor, BWV 1020: II. Adagio",5fBPSa0csCl8hLjZMmA4tm,https://i.scdn.co/image/ab67616d00001e028a5465...,2024-06-10 13:37:31+00:00
29,Johann Sebastian Bach,"Sonata in G Minor, BWV 1020: I. Allegro",1ZtN6evqrtg3YgxBwNNiCs,https://i.scdn.co/image/ab67616d00001e028a5465...,2024-06-10 13:25:17+00:00
28,Johann Sebastian Bach,"Sonata for Violin Solo No. 3 in C, BWV 1005: 4...",67zevEGj7yP12LAJtNPkbT,https://i.scdn.co/image/ab67616d00001e0215d59e...,2024-06-10 13:21:02+00:00
27,Arturo Márquez,Danzón No. 2 (1994),0xVMn4arILsRDGfPRrQbJs,https://i.scdn.co/image/ab67616d00001e029e8558...,2024-06-09 01:15:57+00:00


In [26]:
from skimage import io
import matplotlib.pyplot as plt

def visualize_songs(df):
    """ 
    Visualize cover art of the songs in the inputted DataFrame.

    This function takes a DataFrame containing song details, specifically URLs to cover art images, and visualizes these images in a grid 
    format. Each image is displayed along with the song name as a label below it. The grid is dynamically sized based on the number of songs in the DataFrame.

    Parameters: 
        df (pandas.DataFrame): A DataFrame containing playlist information, including a column 'url' with URLs to the cover art images and a column 'name' with the song names.
    """
    
    # Extract URLs of cover art images from the DataFrame
    temp = df['url'].values

    # Create a figure with a dynamic height based on the number of images
    plt.figure(figsize=(15, int(0.625 * len(temp))))

    # Define the number of columns for the grid layout
    columns = 5
    
    # Loop through each URL and plot the corresponding image
    for index, url in enumerate(temp):
        # Create a subplot for each image
        plt.subplot(int(len(temp) / columns + 1), columns, index + 1)

        # Read the image from the URL
        image = io.imread(url)

        # Display the image
        plt.imshow(image)
        
        # Remove axis ticks for a cleaner look
        plt.xticks(color='w', fontsize=0.1)
        plt.yticks(color='w', fontsize=0.1)

        # Add the song name as a label below the image
        plt.xlabel(df['name'].values[index], fontsize=10)

        # Adjust layout for better spacing
        plt.tight_layout(h_pad=0.4, w_pad=0)
        plt.subplots_adjust(wspace=None, hspace=None)

    # Show the plot
    plt.show()

In [ ]:
visualize_songs(liked_songs)

In [57]:
def generate_playlist_feature_vector(complete_feature_set, playlist_df, decay_factor):
    """ 
    Summarize a user's playlist into a single vector.

    This function processes a user's playlist to create a summarized feature vector that represents the playlist. 
    It applies a recency bias to give more weight to recently added songs. The function also returns a DataFrame of songs that are not 
    in the user's playlist for potential further analysis or recommendations.

    Parameters: 
        complete_feature_set (pandas.DataFrame): DataFrame containing all features for the Spotify songs.
        playlist_df (pandas.DataFrame): DataFrame containing the user's playlist information.
        decay_factor (float): A float value representing the decay_factor for the recency bias. The lower the value, the less importance older songs will have.
        
    Returns: 
        pandas.Series: A single feature vector that summarizes the playlist.
        pandas.DataFrame: A DataFrame containing songs not in the user's playlist.
    """
    
    # Filter the complete feature set to include only the songs in the user's playlist
    complete_feature_set_playlist = complete_feature_set[complete_feature_set['track_id'].isin(playlist_df['track_id'].values)]
    
    # Merge with playlist DataFrame to include the date added information
    complete_feature_set_playlist = complete_feature_set_playlist.merge(playlist_df[['track_id', 'date_added']], on='track_id', how='inner')
    
    # Create a DataFrame containing songs not in the user's playlist
    complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['track_id'].isin(playlist_df['track_id'].values)]
    
    # Sort the playlist feature set by date added in descending order
    playlist_feature_set = complete_feature_set_playlist.sort_values('date_added', ascending=False)

    # Get the date of the most recently added song
    most_recent_date = playlist_feature_set.iloc[0, -1]
    
    # Calculate the number of months from the most recent date for each song in the playlist
    for index, row in playlist_feature_set.iterrows():
        playlist_feature_set.loc[index, 'months_from_recent'] = int((most_recent_date.to_pydatetime() - row['date_added'].to_pydatetime()).days / 30)
        
    # Apply an exponential decay function to set the weights (mimics recency bias)
    playlist_feature_set['weight'] = playlist_feature_set['months_from_recent'].apply(lambda x: decay_factor ** (x))
    
    # Create a weighted version of the playlist feature set
    playlist_feature_set_weighted = playlist_feature_set.copy()
    playlist_feature_set_weighted.update(playlist_feature_set_weighted.iloc[:, :-4].mul(playlist_feature_set_weighted['weight'], 0))
    
    # Exclude the last four columns (which include metadata and weights) to create the final weighted feature set
    playlist_feature_set_weighted_final = playlist_feature_set_weighted.iloc[:, :-4]
    
    # Summarize the playlist into a single vector by summing the weighted features
    return playlist_feature_set_weighted_final.sum(axis=0), complete_feature_set_nonplaylist

Choosing the appropriate value for the decay_factor in the generate_playlist_feature_vector function depends on how much recency bias 
you want to incorporate into your recommendation system. The decay_factor determines how quickly the influence of older songs diminishes 
compared to more recently added songs. Here are some guidelines and considerations to help you choose the right value:

    Value Range:
        The decay_factor should be a value close to 1 but less than 1. Common choices are between 0.8 and 0.99. 
        This range ensures that the decay is not too rapid and maintains a reasonable influence for older songs.

    Recency Emphasis:
        Higher Values - lower recency bias (e.g., 0.95 - 0.99):
            Slower decay of weights.
            Older songs retain more influence.
            Suitable if user preferences change gradually or if older songs are still relevant.
        Lower Values - higher recency bias (e.g., 0.8 - 0.9):
            Faster decay of weights.
            More emphasis on recent songs.
            Suitable if user preferences change rapidly or if you want to prioritize very recent behavior.

    Domain Considerations:
        Consider the typical listening behavior and the context of the playlist. For example, daily playlists might benefit from a 
        higher recency bias compared to long-term favorite collections.

In [58]:
complete_feature_set_playlist_vector, complete_feature_set_nonplaylist = generate_playlist_feature_vector(complete_feature_set, chosen_user_playlist, 0.99)

In [65]:

display(HTML(complete_feature_set_nonplaylist.tail().to_html()))

,genre|acoustic,genre|afrobeat,genre|age,genre|alt,genre|ambient,genre|and,genre|bass,genre|black,genre|blues,genre|breakbeat,genre|cantopop,genre|chicago,genre|chill,genre|classical,genre|club,genre|comedy,genre|country,genre|dance,genre|dancehall,genre|death,genre|deep,genre|detroit,genre|disco,genre|drum,genre|dub,genre|dubstep,genre|edm,genre|electro,genre|electronic,genre|emo,genre|film,genre|folk,genre|forro,genre|french,genre|funk,genre|garage,genre|german,genre|gospel,genre|goth,genre|grindcore,genre|groove,genre|guitar,genre|hard,genre|hardcore,genre|hardstyle,genre|heavy,genre|hip,genre|hop,genre|house,genre|indian,genre|indie,genre|industrial,genre|jazz,genre|metal,genre|metalcore,genre|minimal,genre|new,genre|opera,genre|party,genre|piano,genre|pop,genre|power,genre|progressive,genre|psych,genre|punk,genre|rock,genre|roll,genre|romance,genre|sad,genre|salsa,genre|samba,genre|sertanejo,genre|show,genre|singer,genre|ska,genre|sleep,genre|songwriter,genre|soul,genre|spanish,genre|swedish,genre|tango,genre|techno,genre|trance,genre|trip,genre|tunes,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,pop|0,pop|1,pop|2,pop|3,pop|4,pop|5,pop|6,pop|7,pop|8,pop|9,pop|10,pop|11,pop|12,pop|13,pop|14,pop|15,pop|16,pop|17,pop|18,pop|19,pop|20,year|2000,year|2001,year|2002,year|2003,year|2004,year|2005,year|2006,year|2007,year|2008,year|2009,year|2010,year|2011,year|2012,year|2013,year|2014,year|2015,year|2016,year|2017,year|2018,year|2019,year|2020,year|2021,year|2022,year|2023,track_id
1159759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.64552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.763743,0.0,0.075126,0.1484,0.160714,0.015160,0.065261,0.000028,0.03180,0.10440,0.086363,0.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0m27F0IGHLGAWhqd6ccYst
1159760,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.64552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.763743,0.0,0.103927,0.1350,0.157182,0.006715,0.158233,0.000026,0.02600,0.05280,0.095920,0.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6er9p611eHEcUCU50j7D57
1159761,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.64552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.763743,0.0,0.098892,0.0880,0.154307,0.005644,0.095783,0.000626,0.01872,0.00702,0.080063,0.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7jsMMqxy1tt0rH5FzYcZTQ
1159762,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.64552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.763743,0.0,0.096677,0.0810,0.139274,0.005685,0.086546,0.000013,0.02500,0.04040,0.107111,0.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [88]:
test_feature_set_playlist_vector, test_feature_set_nonplaylist = generate_playlist_feature_vector(test_feature_set, chosen_user_playlist, 0.99)

In [89]:
test_feature_set_nonplaylist.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_id
0,0.097281,0.0606,0.149496,0.008836,0.139357,0.000000,0.02300,0.0278,0.106728,53QF56cjZA9RTuuMZDrSA6
1,0.115206,0.0908,0.148786,0.005314,0.095783,0.000003,0.01948,0.1030,0.112149,1s8tP3jP4GZcyHDsjvw218
2,0.082377,0.0468,0.138129,0.006653,0.067871,0.000010,0.01790,0.0290,0.111869,7BRCa8MPiyuvr2VU3O9W0F
3,0.078953,0.0502,0.150159,0.007477,0.162048,0.000000,0.01594,0.1016,0.163973,63wsZUhUZLlh1OsyrZq7sz
4,0.086606,0.1582,0.163931,0.006220,0.014578,0.003860,0.02200,0.0434,0.137495,6nXIYClvJAfi6ujLiKqEq8


In [78]:
test_feature_set_playlist_vector


danceability        3.810431
energy              6.271520
loudness            6.284883
speechiness         0.478177
acousticness        0.482392
instrumentalness    1.046652
liveness            1.658957
valence             3.842394
tempo               4.002748
dtype: float64

Song Score Calculation

The playlist vectors will be compared to individual song vectors using cosine similarity.

In [79]:
def generate_recommendations(spotify_data, playlist_features, nonplaylist_features):
    """ 
    Generate song recommendations based on a user's playlist.

    This function calculates the cosine similarity between the user's playlist feature vector and the feature vectors of all songs not in the playlist. 
    It then sorts the songs based on their similarity to the playlist and returns the top recommendations.

    Parameters: 
        spotify_data (pandas.DataFrame): DataFrame containing all features for the Spotify songs in the Kaggle data.
        playlist_features (pandas.Series): A single feature vector that summarizes the user's playlist.
        nonplaylist_features (pandas.DataFrame): DataFrame containing songs not in the user's playlist.
        
    Returns: 
        pandas.DataFrame: A DataFrame containing the top song recommendations based on the user's playlist.
    """
    def get_track_info(track_id):
        try:
            track_info = sp.track(track_id)
            return track_info['album']['images'][1]['url'] if len(track_info['album']['images']) > 1 else None
        except (IndexError, KeyError, spotipy.SpotifyException):
            return None

    nonplaylist_recommendations = spotify_data[spotify_data['track_id'].isin(nonplaylist_features['track_id'].values)]
    # playlist_features.values.reshape(1, -1):
    # This part converts the playlist_features Series to a NumPy array and reshapes it to be a 2D array with one row and as many columns as there are features.
    # .reshape(1, -1) is necessary because cosine_similarity expects 2D arrays as input.
    nonplaylist_recommendations['similarity'] = cosine_similarity(nonplaylist_features.drop('track_id', axis=1).values, playlist_features.values.reshape(1, -1))[:,0]
    nonplaylist_recommendations_top = nonplaylist_recommendations.sort_values('similarity', ascending=False).head(300)
    nonplaylist_recommendations_top['url'] = nonplaylist_recommendations_top['track_id'].apply(lambda x: get_track_info(x))
    nonplaylist_recommendations_top = nonplaylist_recommendations_top.rename(columns={'track_name': 'name'})

    return nonplaylist_recommendations_top

In [30]:
top_recommendations = generate_recommendations(spotify_df, complete_feature_set_playlist_vector, complete_feature_set_nonplaylist)

In [90]:
non_genre_top_recommendations = generate_recommendations(spotify_df, test_feature_set_playlist_vector, test_feature_set_nonplaylist)

In [91]:
non_genre_top_recommendations.head()

,Unnamed: 0,artist_name,name,track_id,popularity,popularity_transform,year,genre,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,similarity,url
1045327,1293575,Søren Bødker Madsen,Tuxens Romance,0DGxYkdUvdH8iChF1j6YSK,1,0,2009,guitar,0.373,0.220,...,0.0445,0.973,0.862,0.128,0.442,114.243,134253,4,0.999403,https://i.scdn.co/image/ab67616d00001e02728d41...
738153,834879,Greg Maroney,I'll Remember You,60rbl1CtES3CEkEUJ5dhV5,1,0,2002,new-age,0.367,0.183,...,0.0434,0.994,0.888,0.128,0.490,123.277,199747,4,0.999367,https://i.scdn.co/image/ab67616d00001e02e7d295...
459479,459479,Michele McLaughlin,Lost,0WgHSXZ79qclgPnW2WHEYQ,9,1,2020,new-age,0.354,0.169,...,0.0514,0.988,0.904,0.117,0.492,114.279,217933,4,0.999201,https://i.scdn.co/image/ab67616d00001e02c8e6ec...
1044721,1292821,Søren Bødker Madsen,Hallelujah,7f0c0Ge1JBRGPaGdKqMnyX,14,2,2009,guitar,0.363,0.187,...,0.0425,0.971,0.900,0.103,0.469,111.548,181907,1,0.999139,https://i.scdn.co/image/ab67616d00001e02728d41...
696432,776142,Lara Downes,Suite from the Plow That Broke the Plains: 2. ...,4aCPx2BVXj77iUHf0fT9bM,0,0,2001,piano,0.383,0.163,...,0.0403,0.995,0.888,0.116,0.527,118.186,104360,3,0.999113,https://i.scdn.co/image/ab67616d00001e02bfd8fc...


In [ ]:
visualize_songs(top_recommendations)

In [82]:
#  user_playlist_create(user, name, public=True, collaborative=False, description='')
def create_playlist_from_recommendations(playlist_name, recommendations, batch_size=100):
    """
    Add tracks to a Spotify playlist in batches.

    Parameters:
        sp (spotipy.Spotify): Spotipy client instance.
        playlist_id (str): The ID of the playlist.
        track_ids (list): List of track IDs to add to the playlist.
        batch_size (int): Number of tracks to add per batch. Maximum is 100 per call.

    Returns:
        None
    """

    position = 0
    userId = sp.me()['id']
    new_playlist = sp.user_playlist_create(userId, playlist_name, public=False, collaborative=False, description='Playlist created by the experimental recommender system')
    track_ids = recommendations.get('track_id').values
    
    # print(f"Adding {len(track_ids)} tracks to the playlist...")

    for i in range(0, len(track_ids), batch_size):
        batch = track_ids[i:i + batch_size]
        sp.playlist_add_items(new_playlist['id'], batch, position=position)
        position += len(batch)
    



In [92]:
create_playlist_from_recommendations('Fresh Nongenre Classical Based', non_genre_top_recommendations)